# Direct Preference Optimization (DPO)原理详解及公式推导

#### 文章目录

* [1. 概述](#1-概述)
* [2. 几个关键的预备知识](#2-几个关键的预备知识)
    * [2.1 Kullback-Leibler 散度（KL 散度）](#21-kullback-leibler-散度kl-散度)
        * [2.1.1 KL 散度的定义](#211-kl-散度的定义)
        * [2.1.2 KL 散度的性质](#212-kl-散度的性质)
        * [2.1.3 KL 例子](#213-kl-例子)
        * [2.1.4 在DPO中的应用](#214-在dpo中的应用)
    * [2.2 Bradley-Terry 模型](#22-bradley-terry-模型)
        * [2.2.1 BT 模型定义](#221-bt-模型定义)
        * [2.2.2 参数估计](#222-参数估计)
        * [2.2.3 BT 例子](#223-bt-例子)
* [3. DPO 的推导过程](#3-dpo-的推导过程)
    * [3.1 优化目标推导](#31-优化目标推导)
        * [3.1.1 奖励模型损失](#311-奖励模型损失)
        * [3.1.2 优化目标](#312-优化目标)
    * [3.2 损失函数](#32-损失函数)
    * [3.3 DPO 梯度更新](#33-dpo-梯度更新)
    * [3.4 DPO 主要流程](#34-dpo-主要流程)
* [4. DPO 数据集](#4-dpo-数据集)
* [5. 总结](#5-总结)

## 1. 概述

Direct Preference Optimization (DPO) 是一种专为大型语言模型（LLMs）设计的训练方法，旨在通过人类偏好数据来优化模型，而无需使用复杂的强化学习算法（如Proximal Policy Optimization, PPO）。DPO 的核心思想是通过偏好数据直接调整模型参数，绕过显式奖励模型的拟合以及复杂的强化学习优化过程。这种简化的训练方法不仅提高了训练效率，还避免了传统 RLHF 方法中常见的不稳定性。

DPO 的亮点在于：

（1）直接使用最大似然估计（Maximum Likelihood Estimation, MLE）来优化策略；

（2）在无需显式拟合奖励模型的情况下，能够高效地学习出与人类偏好一致的最优策略；

（3）相较于 RLHF，DPO 训练更加稳定和简化，并且具有与之相当的性能。

本文主要介绍 DPO 的原理，以及理论公式是如何推导的。

## 2. 几个关键的预备知识

DPO 涉及到 KL 散度、Bradley-Terry 模型等预备知识，预备知识是基础，对于后续公式推导的理解至关重要，下面会详细介绍。如果熟悉这部分知识可以跳过这部分。

### 2.1 Kullback-Leibler 散度（KL 散度）

**Kullback-Leibler 散度（KL 散度）** ，又称为**相对熵** ，是信息论中的一个重要概念。它用于衡量两个概率分布之间的差异，特别是衡量用一个分布  $Q$ 去近似另一个目标分布  $P$ 时的效率损失。

KL 散度可以理解为两种分布之间的“信息差异”。具体而言，KL 散度衡量的是用分布  $Q$ 来编码分布  $P$ 所需要的额外信息量。假设分布  $P(x)$ 是我们想要捕捉的真实分布，而我们用分布  $Q(x)$ 来表示它。如果  $Q(x)$ 偏离  $P(x)$，我们在编码时会付出信息损失。

KL 散度值越小，表示两个分布越接近。

#### 2.1.1 KL 散度的定义

KL 散度表示两个概率分布  $P(x)$ 和  $Q(x)$ 之间的“距离”或差异程度，定义公式为：

$D_{\text{KL}}(P \| Q) = \sum_x P(x) \log \frac{P(x)}{Q(x)}$

或者在连续情形下：

$D_{\text{KL}}(P \| Q) = \int P(x) \log \frac{P(x)}{Q(x)} dx$

其中：

* $P(x)$：目标分布（真实分布），表示事件  $x$ 发生的概率。
* $Q(x)$：近似分布（模型分布），表示用来近似  $P(x)$ 的概率。

KL 散度衡量的是，如果  $P(x)$ 是“真实”的概率分布，而我们使用  $Q(x)$ 来表示  $P(x)$，我们将损失多少信息。

#### 2.1.2 KL 散度的性质

KL 散度具有以下重要性质：

（1）**非负性** ：$D_{\text{KL}}(P \| Q) \geq 0$，且当且仅当  $P(x) = Q(x)$ 时，$D_{\text{KL}}(P \| Q) = 0$。这意味着两个分布完全一致时，KL 散度为零。

（2）**非对称性** ：$D_{\text{KL}}(P \| Q) \neq D_{\text{KL}}(Q \| P)$。KL 散度并不是衡量两个分布对称差异的度量，所以  $D_{\text{KL}}(P \| Q)$ 和  $D_{\text{KL}}(Q \| P)$ 的值可能不同。

#### 2.1.3 KL 例子

**例子 1：硬币投掷**
假设我们有两个硬币，硬币 A 和硬币 B，它们的投掷概率分别是：

* 硬币 A：$P(正面) = 0.8$， $P(反面) = 0.2$
* 硬币 B：$Q(正面) = 0.6$， $Q(反面) = 0.4$

现在，我们希望用硬币 B（分布  $Q$）来近似硬币 A（分布  $P$）。我们可以计算 KL 散度：

$D_{\text{KL}}(P \| Q) = P(正面) \log \frac{P(正面)}{Q(正面)} + P(反面) \log \frac{P(反面)}{Q(反面)}$

代入数值：
$D_{\text{KL}}(P \| Q) = 0.8 \log \frac{0.8}{0.6} + 0.2 \log \frac{0.2}{0.4}$

$D_{\text{KL}}(P \| Q) \approx 0.8 \times 0.222 + 0.2 \times (-0.301) \approx 0.109$

这个值越小，表示两个分布越接近；而越大，则表示它们的差异越大。在这个例子中，KL 散度表示硬币 B 与硬币 A 之间的差异。

#### 2.1.4 在DPO中的应用

在 DPO 中，KL 散度是一个关键组成部分，用于限制模型的训练。DPO 的优化目标是最大化模型生成优选输出的概率，同时最小化模型与参考模型之间的 KL 散度。具体来说，DPO 的目标函数为（后面会详细介绍，这里暂时了解）：

$\max_{\pi_\theta} \mathbb{E}_{x \sim D, y \sim \pi_\theta(y | x)} [r_\varphi(x, y)] - \beta D_{\text{KL}}(\pi_\theta(y | x) \| \pi_{\text{ref}}(y | x))$

其中：

* $D_{\text{KL}}$ 表示当前模型分布  $\pi_\theta$ 和参考模型  $\pi_{\text{ref}}$ 之间的差异。
* $\beta$ 是控制 KL 散度影响的超参数。

通过最小化 KL 散度，DPO 确保了当前模型在优化过程中不会偏离参考策略太远，从而保持模型的稳定性。

### 2.2 Bradley-Terry 模型

Bradley-Terry 模型是一种用来比较多个对象（比如队伍、产品或选项）之间相对“强度”的方法。这个模型可以帮助我们根据一系列对比结果（例如比赛胜负、用户偏好等）来估计每个对象的“实力值”。它被广泛用于体育比赛预测、推荐系统和偏好分析等领域。

#### 2.2.1 模型的基本思想

假设有一组对象，比如 4 个队伍：A、B、C 和 D。每个对象都有一个“实力值”，用一个数字（比如 `λ`）表示。实力值越大，代表这个对象越强。

如果两个对象进行比较，比如队伍 A 和队伍 B 比赛，A 获胜的概率可以用以下公式计算：

**A 获胜的概率**：

$$
P(A > B) = \frac{\lambda_A}{\lambda_A + \lambda_B}
$$

这个公式的意思是：A 获胜的概率等于 A 的实力值除以 A 和 B 实力值之和。

**例子**：  
如果队伍 A 的实力值是 1，队伍 B 的实力值是 3，那么 A 获胜的概率是：

$$
P(A > B) = \frac{1}{1 + 3} = 0.25
$$

这意味着，A 获胜的概率是 25%，而 B 获胜的概率是 75%。

#### 2.2.2 如何估计每个对象的实力值（最大似然估计）

在实际问题中，我们并不知道每个对象的实力值（比如 `λ_A`, `λ_B` 等），需要通过比赛结果来推算出这些值。Bradley-Terry 模型使用的是一种叫**最大似然估计**的方法。

**最大似然估计的想法**：  
最大似然估计的目标是找到一组实力值，使得这些比赛结果的“发生概率”最大。换句话说，我们希望实力值的选择尽可能符合已有的比赛数据。

**似然函数**：  
假设我们有很多场比赛的结果，比如：

- A vs B，A 胜
- A vs C，C 胜
- B vs D，B 胜

我们可以用胜率公式来表示每场比赛发生的概率。例如：

- A vs B，A 的胜率是 $P(A > B) = \frac{\lambda_A}{\lambda_A + \lambda_B}$；
- A vs C，C 的胜率是 $P(C > A) = \frac{\lambda_C}{\lambda_A + \lambda_C}$。

如果我们把所有比赛的概率乘起来，就能得到一个公式，表示这些比赛结果的“总概率”，叫做**似然函数**。公式如下：

$$
L(\lambda_A, \lambda_B, \lambda_C, \lambda_D) = \prod_{(i, j) \in \text{pairs}} \left( \frac{\lambda_i}{\lambda_i + \lambda_j} \right)^{x_{ij}} \left( \frac{\lambda_j}{\lambda_i + \lambda_j} \right)^{1-x_{ij}}
$$

- $x_{ij} = 1$ 表示 i 赢了 j；
- $x_{ij} = 0$ 表示 i 输了 j。

我们需要调整每个对象的实力值（`λ_A`, `λ_B` 等），让这个似然函数的值最大化。

#### 2.2.3 一个简单例子

假设我们有一个小型体育联赛，4 支队伍 A、B、C 和 D 进行了以下比赛：

- A vs B：A 胜
- A vs C：C 胜
- A vs D：A 胜
- B vs C：B 胜
- B vs D：B 胜
- C vs D：C 胜

我们希望用 Bradley-Terry 模型来估计每支队伍的实力值（`λ_A`, `λ_B`, `λ_C`, `λ_D`）。

**步骤 1：用模型表示胜率**  

根据 Bradley-Terry 模型，两个队伍 i 和 j 的胜率可以表示为：

$$
P(i > j) = \frac{\lambda_i}{\lambda_i + \lambda_j}
$$

这意味着，如果我们知道每个队伍的实力值，就能计算它们之间的胜率。

**步骤 2：写出似然函数**  

根据比赛结果，似然函数可以写成所有比赛概率的乘积。例如：

- A vs B，A 胜，对应概率是 $\frac{\lambda_A}{\lambda_A + \lambda_B}$；
- A vs C，C 胜，对应概率是 $\frac{\lambda_C}{\lambda_A + \lambda_C}$；
- 其他比赛类似。

所以似然函数是：

$$
L(\lambda_A, \lambda_B, \lambda_C, \lambda_D) = \left( \frac{\lambda_A}{\lambda_A + \lambda_B} \right) \left( \frac{\lambda_C}{\lambda_A + \lambda_C} \right) \left( \frac{\lambda_A}{\lambda_A + \lambda_D} \right) \left( \frac{\lambda_B}{\lambda_B + \lambda_C} \right) \left( \frac{\lambda_B}{\lambda_B + \lambda_D} \right) \left( \frac{\lambda_C}{\lambda_C + \lambda_D} \right)
$$

**步骤 3：简化问题**  

为了简化计算，我们可以固定一个队伍的实力值，比如设定 `λ_A = 1`。这样，我们只需要估计其他 3 个参数：`λ_B`, `λ_C`, `λ_D`。

**步骤 4：最大化似然函数**  

通过数学方法（比如优化算法），我们可以找到使似然函数最大的参数值。最终可能得到以下结果：

- `λ_A = 1`（固定值）
- `λ_B = 1.5`
- `λ_C = 2.0`
- `λ_D = 1.2`

**步骤 5：解释实力值**  

从计算结果可以看出：

- 队伍 C 的实力值最高（`λ_C = 2.0`），说明它最强。
- 队伍 B 实力次之（`λ_B = 1.5`）。
- 队伍 D 比较弱（`λ_D = 1.2`）。
- 队伍 A 最弱（`λ_A = 1`）。

**步骤 6：计算胜率**  

现在，我们可以用这些实力值计算任意两队之间的胜率。例如：

- A 对 B 的胜率：
  
$$
P(A > B) = \frac{\lambda_A}{\lambda_A + \lambda_B} = \frac{1}{1 + 1.5} = \frac{1}{2.5} = 0.4
$$

这意味着 A 对 B 的胜率是 40%。

- C 对 D 的胜率：

$$
P(C > D) = \frac{\lambda_C}{\lambda_C + \lambda_D} = \frac{2.0}{2.0 + 1.2} = \frac{2.0}{3.2} \approx 0.625
$$

这意味着 C 对 D 的胜率是 62.5%。

---

通过最大似然估计，Bradley-Terry 模型可以根据比赛结果推断出每支队伍的实力值，并预测它们之间的胜率。这种方法也可以用在其他领域，比如预测用户更喜欢哪个产品，或者对候选人进行偏好排序等。

## 3. DPO 的推导过程

### 3.1 优化目标推导

#### 3.1.1 奖励模型损失

设  $y_w$ 是人类偏好中的优选，$y_l$ 是次优选，根据 BT 模型人类偏好  $y_w$ 优于  $y_l$ 的概率为：

$$
P(y_w \succ y_l|x)=\frac{\lambda_w}{\lambda_w + \lambda_l}
$$

那么在强化模型中，我们用什么来表示  $\lambda$ 呢，用的是奖励模型的分数  $r(x,y)$，考虑到  $r(x,y)$ 可能为负，所以用指数函数表示即  $e^{r(x,y)}$，那么上式可以写成：

$$
P(y_w \succ y_l|x)=\frac{e^{r(x,y_w)}}{e^{r(x,y_w)}+e^{r(x,y_l)}}
$$

上式分子分母同时除以  $e^{r(x,y_w)}$，得到：

$$
P(y_w \succ y_l|x)=\frac{1}{1+e^{r(x,y_l)-r(x,y_w)}}=\sigma({r(x,y_w)-r(x,y_l)})
$$

其中  $\sigma$ 是 logistic 函数：

$$
\sigma(x)=\frac{1}{1+e^{-x}}
$$

对于所有的数据对  $\mathcal{D}= \{ (x^{(i)},y_\omega^{(i)},y_l^{(i)}) \}_{i=1}^N$，我们根据极大似然估计：

$$
\mathcal{L(r,\mathcal{D})}=\prod_{(x,y_{\omega},y_l)\sim\mathcal{D}} \sigma({r(x,y_w)-r(x,y_l)})^n
$$

然后对等式两边同时取对数、取负数、取平均值，原先的极大似然估计就转换成了负对数似然损失（negative log-likelihood loss）。以下是具体推导过程：

1. 对等式两边取对数：

   $$
   \log \mathcal{L}(r, \mathcal{D}) = \sum_{(x, y_w, y_l) \sim \mathcal{D}} \log \sigma(r(x, y_w) - r(x, y_l))
   $$

2. 再取负数：

   $$
   -\log \mathcal{L}(r, \mathcal{D}) = -\sum_{(x, y_w, y_l) \sim \mathcal{D}} \log \sigma(r(x, y_w) - r(x, y_l))
   $$

3. 最后取平均值（对整个数据集进行标准化）：

   $$
   \mathcal{L_R(r_{\phi},\mathcal{D})} = -\mathbb{E}_{(x, y_w, y_l) \sim \mathcal{D}} \log \sigma(r_{\phi}(x, y_w) - r_{\phi}(x, y_l))
   $$

最终得到负对数似然损失的形式：

$$
\mathcal{L_R(r_{\phi},\mathcal{D})}=-\mathbb{E}_{(x,y_{\omega},y_l)\sim\mathcal{D}} \log \sigma({r_{\phi}(x,y_w)-r_{\phi}(x,y_l)})
$$

其中：

* $\sigma$ 是 logistic 函数
* $r_{\phi}(x,y)$ 是隐式奖励模型


以下是对 DPO 优化目标公式推导的细化解释：

---

### 3.1.2 优化目标推导详解

#### 初始目标函数
DPO 的优化目标可表示为：
$$
\max_{\pi_\theta} \mathbb{E}_{x \sim D, y \sim \pi_\theta(y | x)} [r_\phi(x, y)] - \beta D_{\text{KL}}(\pi_\theta(y | x) \| \pi_{\text{ref}}(y | x)),
$$
其中第一项最大化隐式奖励的期望，第二项通过 KL 散度约束策略 $\pi_\theta$ 与参考策略 $\pi_{\text{ref}}$ 的偏离。

---

#### 推导步骤分解

**1. 展开 KL 散度**
KL 散度的定义式为：
$$
D_{\text{KL}}(\pi \| \pi_{\text{ref}}) = \mathbb{E}_{y \sim \pi(y|x)} \left[ \log \frac{\pi(y|x)}{\pi_{\text{ref}}(y|x)} \right].
$$
将其代入目标函数，得到：
$$
\max_{\pi} \mathbb{E}_{x \sim D} \left[ \mathbb{E}_{y \sim \pi(y|x)} [r(x,y)] - \beta \mathbb{E}_{y \sim \pi(y|x)} \log \frac{\pi(y|x)}{\pi_{\text{ref}}(y|x)} \right].
$$

**2. 合并期望项**
由于两个期望共享相同的分布 $x \sim D, y \sim \pi(y|x)$，可合并为单一期望：
$$
\max_{\pi} \mathbb{E}_{x \sim D} \mathbb{E}_{y \sim \pi(y|x)} \left[ r(x,y) - \beta \log \frac{\pi(y|x)}{\pi_{\text{ref}}(y|x)} \right].
$$

**3. 转换为最小化问题**
通过引入负号，最大化问题等价于最小化：
$$
\min_{\pi} \mathbb{E}_{x \sim D} \mathbb{E}_{y \sim \pi(y|x)} \left[ \log \frac{\pi(y|x)}{\pi_{\text{ref}}(y|x)} - \frac{1}{\beta} r(x,y) \right].
$$

**4. 引入归一化因子 $Z(x)$**
定义归一化因子：
$$
Z(x) = \sum_y \pi_{\text{ref}}(y|x) e^{\frac{1}{\beta} r(x,y)},
$$
并构造辅助分布：
$$
\pi^*(y|x) = \frac{1}{Z(x)} \pi_{\text{ref}}(y|x) e^{\frac{1}{\beta} r(x,y)}.
$$
此时，$\pi^*$ 是一个合法的概率分布（因已归一化），且与 $\pi$ 无关。

**5. 重写目标函数**
将 $\pi_{\text{ref}}(y|x) e^{\frac{1}{\beta} r(x,y)}$ 替换为 $Z(x) \pi^*(y|x)$，目标函数变为：
$$
\min_{\pi} \mathbb{E}_{x \sim D} \mathbb{E}_{y \sim \pi(y|x)} \left[ \log \frac{\pi(y|x)}{\pi^*(y|x)} - \log Z(x) \right].
$$
注意到 $\log Z(x)$ 与 $\pi$ 无关，可分离出优化目标：
$$
\min_{\pi} \mathbb{E}_{x \sim D} \left[ \underbrace{\mathbb{E}_{y \sim \pi(y|x)} \log \frac{\pi(y|x)}{\pi^*(y|x)}}_{D_{\text{KL}}(\pi \| \pi^*)} - \log Z(x) \right].
$$

**6. 最优解分析**
KL 散度 $D_{\text{KL}}(\pi \| \pi^*)$ 的最小值为 0，当且仅当 $\pi = \pi^*$。因此，最优策略为：
$$
\pi_{\text{opt}}(y|x) = \pi^*(y|x) = \frac{1}{Z(x)} \pi_{\text{ref}}(y|x) e^{\frac{1}{\beta} r(x,y)}.
$$

---

#### 关键推导点总结

1. **KL 散度的作用**  
   KL 项作为正则化项，确保策略 $\pi_\theta$ 不会过度偏离参考模型 $\pi_{\text{ref}}$，避免过度优化奖励导致模型崩溃。

2. **隐式奖励与策略的关系**  
   奖励函数 $r(x,y)$ 被隐式编码到策略 $\pi_\theta$ 中，通过指数加权调整参考策略的概率分布。

3. **归一化因子 $Z(x)$ 的意义**  
   $Z(x)$ 确保 $\pi^*$ 是有效概率分布，其值依赖于 $x$ 和奖励函数，但在优化过程中可忽略（因与 $\pi$ 无关）。

4. **闭式解的存在性**  
   通过变分法直接得到最优策略的解析形式，避免了复杂的强化学习训练流程（如 PPO 中的策略-值函数交替优化）。

---

#### 公式关联性图示

$$
\begin{aligned}
\text{初始目标} &\rightarrow \text{KL 散度展开} \rightarrow \text{合并期望} \rightarrow \text{符号转换} \\
&\rightarrow \text{引入辅助分布 } \pi^* \rightarrow \text{分离无关项 } \log Z(x) \rightarrow \text{最优闭式解}.
\end{aligned}
$$


### 3.2 损失函数推导详解

---

#### **从优化目标到隐式奖励的表达式**
根据 3.1.2 节的推导，最优策略 $\pi_\theta(y|x)$ 的闭式解为：
$$
\pi_\theta(y|x) = \frac{1}{Z(x)} \pi_{\text{ref}}(y|x) e^{\frac{1}{\beta} r(x,y)},
$$
其中 $Z(x) = \sum_y \pi_{\text{ref}}(y|x) e^{\frac{1}{\beta} r(x,y)}$ 是归一化因子。对该式取对数并整理，可解出隐式奖励函数 $r(x,y)$：
$$
r(x,y) = \beta \log \frac{\pi_\theta(y|x)}{\pi_{\text{ref}}(y|x)} + \beta \log Z(x).
$$

---

#### **基于 Bradley-Terry 模型的偏好概率**
在偏好数据中，假设优胜输出 $y_w$ 优于劣质输出 $y_l$ 的概率由 Bradley-Terry 模型定义：
$$
P(y_w \succ y_l | x) = \frac{1}{1 + e^{r(x,y_l) - r(x,y_w)}}.
$$
将 $r(x,y)$ 的表达式代入，得到：
$$
P(y_w \succ y_l | x) = \frac{1}{1 + \exp\left( \beta \log \frac{\pi_\theta(y_l|x)}{\pi_{\text{ref}}(y_l|x)} - \beta \log \frac{\pi_\theta(y_w|x)}{\pi_{\text{ref}}(y_w|x)} \right)}.
$$

---

#### **损失函数的构建**
为训练策略 $\pi_\theta$，需最大化偏好数据的对数似然：
$$
\mathcal{L}(\pi_\theta) = \mathbb{E}_{(x,y_w,y_l) \sim D} \left[ \log P(y_w \succ y_l | x) \right].
$$
将 $P(y_w \succ y_l | x)$ 表达式代入，并取负号转换为最小化问题：
$$
\mathcal{L}_{\text{DPO}}(\pi_\theta; \pi_{\text{ref}}) = -\mathbb{E}_{(x,y_w,y_l) \sim D} \left[ \log \sigma\left( \beta \log \frac{\pi_\theta(y_w|x)}{\pi_{\text{ref}}(y_w|x)} - \beta \log \frac{\pi_\theta(y_l|x)}{\pi_{\text{ref}}(y_l|x)} \right) \right],
$$
其中 $\sigma(z) = \frac{1}{1 + e^{-z}}$ 是 Sigmoid 函数。

---

#### **关键推导步骤说明**

1. **归一化因子 $Z(x)$ 的消除**  
   在奖励函数 $r(x,y)$ 中，$\beta \log Z(x)$ 项仅依赖于 $x$，与 $y$ 无关。在计算奖励差 $r(x,y_l) - r(x,y_w)$ 时，$\beta \log Z(x)$ 会被抵消：
   $$
   r(x,y_l) - r(x,y_w) = \beta \left( \log \frac{\pi_\theta(y_l|x)}{\pi_{\text{ref}}(y_l|x)} - \log \frac{\pi_\theta(y_w|x)}{\pi_{\text{ref}}(y_w|x)} \right).
   $$
   因此，$Z(x)$ 无需显式计算。

2. **损失函数的直观解释**  
   损失函数通过 Sigmoid 函数强制策略 $\pi_\theta$ 满足：
   $$
   \beta \log \frac{\pi_\theta(y_w|x)}{\pi_{\text{ref}}(y_w|x)} \gg \beta \log \frac{\pi_\theta(y_l|x)}{\pi_{\text{ref}}(y_l|x)},
   $$
   即优化目标为：
   - 提升优胜输出 $y_w$ 的概率比 $\frac{\pi_\theta(y_w|x)}{\pi_{\text{ref}}(y_w|x)}$；
   - 抑制劣质输出 $y_l$ 的概率比 $\frac{\pi_\theta(y_l|x)}{\pi_{\text{ref}}(y_l|x)}$。

---

#### **损失函数的意义与特性**

| 特性 | 说明 |
|-------|-------|
| **直接优化偏好** | 通过概率比的对数差，直接对齐人类偏好，无需显式奖励模型。 |
| **隐式正则化** | $\pi_{\text{ref}}$ 作为锚点，通过概率比约束策略更新范围，防止偏离初始策略。 |
| **计算高效** | 仅需策略模型 $\pi_\theta$ 和参考模型 $\pi_{\text{ref}}$ 的概率输出，无需强化学习中的价值函数或重要性采样。 |

---

#### **公式关联性图示**
$$
\begin{aligned}
\text{优化目标闭式解} &\rightarrow \text{隐式奖励表达式} \xrightarrow{\text{Bradley-Terry 模型}} \text{偏好概率} \\
&\xrightarrow{\text{负对数似然}} \text{最终损失函数}.
\end{aligned}
$$

---

#### **与经典强化学习的对比**

| 方法 | 优化目标 | 计算复杂度 | 稳定性 |
|-------|-------|-------|-------|
| **传统 RLHF** | 最大化奖励期望，约束 KL 散度 | 高（需策略-值函数交替优化） | 依赖奖励模型设计 |
| **DPO** | 直接优化策略概率比 | 低（仅需策略模型前向计算） | 隐式正则化，避免策略崩溃 |

---

通过这一损失函数，DPO 将复杂的强化学习问题转化为监督学习框架下的对比优化，显著降低了训练难度和计算成本。



### 3.3 DPO 梯度更新

DPO 的梯度更新通过以下公式计算：

$\nabla_\theta \mathcal{L}_{DPO}(\pi_\theta; \pi_{\text{ref}}) = -\beta \mathbb{E}_{(x, y_w, y_l) \sim D}\left[\sigma(\hat{r}_\theta(x, y_l) - \hat{r}_\theta(x, y_w)) \left(\nabla_\theta \log \pi_\theta(y_w | x) - \nabla_\theta \log \pi_\theta(y_l | x)\right)\right]$

其中，  $\hat{r}_\theta(x, y)$ 是隐式奖励函数，定义为：

$\hat{r}_\theta(x, y) = \beta \log \frac{\pi_\theta(y|x)}{\pi_{\text{ref}}(y|x)}$


其中有2部分：

1. $\nabla_\theta \log \pi_\theta(y_w | x) - \nabla_\theta \log \pi_\theta(y_l | x)$ 表示对于一个策略，要偏向于增加出现  $y_w$ 的可能性，降低出现  $y_l$ 的可能性；
2. $\sigma(\hat{r}_\theta(x, y_l) - \hat{r}_\theta(x, y_w))$ 表示当奖励模型  $r$ 评估出现错误时，可获得更高的权重

梯度更新的直观理解是增加优胜者的生成概率，减少劣胜者的生成概率。

### 3.4 DPO 主要流程

DPO 的一般流程如下：
（1）对于每一个 prompt  $x$，采样完成  $y_1, y_2 \sim \pi_{\text{ref}}(\cdot|x)$，用人类偏好标注以构建离线偏好数据集  $D=\\{(x^{(i)}, y_w^{(i)}, y_l^{(i)})\\}_{i=1}^{N}$
（2）给定  $\pi_{\text{ref}}$ 和  $D$ 以及所需的  $\beta$，优化语言模型  $\pi_{\theta}$（即策略），使其最小化  $\mathcal{L}_{\text{DPO}}$。

> 在实践中，人们倾向于用公开可用的偏好数据集，而不是生成样本和收集人类偏好。 因为偏好数据集是使用  $\pi^{\text{SFT}}$ 采样的，因此我们尽可能初始化  $\pi_{\text{ref}}$。 但是，当  $\pi^{\text{SFT}}$ 不可用时，我们通过最大化首选完成  $(x, y_w)$ 的似然性来初始化  $\pi_{\text{ref}}$，即：
> $\pi_{\text{ref}} = \arg \max_{\pi} \mathbb{E}_{x,y_w \sim D}[\log \pi(y_w|x)]$
> 此过程有助于缓解真实参考分布（不可用）与 DPO 使用的  $\pi_{\text{ref}}$ 之间的分布偏移。

## 4. DPO 数据集

DPO 主要用于有明确偏好数据的任务，如文本生成、对话生成、摘要生成等。DPO 期望数据集具有非常特定的格式，由三部分组成：

* prompt。上下文输入
* chosen。相应的选择响应
* rejected。相应的否定（拒绝）响应

例如：

```
dpo_dataset_dict = {
  "prompt": [
    "hello",
    "how are you",
    "What is your name?",
    "What is your name?",
    "Which is the best programming language?",
    "Which is the best programming language?",
    "Which is the best programming language?",
  ],
  "chosen": [
    "hi nice to meet you",
    "I am fine",
    "My name is Mary",
    "My name is Mary",
    "Python",
    "Python",
    "Java",
  ],
  "rejected": [
    "leave me alone",
    "I am not fine",
    "Whats it to you?",
    "I dont have a name",
    "Javascript",
    "C++",
    "C++",
  ],
}
```

更具体的数据参考 huggingface 数据集 [Anthropic/hh-rlhf](https://huggingface.co/datasets/Anthropic/hh-rlhf)

## 5. 总结

DPO 是一种通过人类偏好数据优化大型语言模型的有效方法。与传统的 RLHF 方法相比，DPO 提供了一个更简单、稳定的训练过程，通过最大似然估计直接优化模型，使其生成的输出更加符合人类偏好。
```


In [ ]:
import torch
import torch.nn.functional as F
from transformers import LlamaForCausalLM, LlamaConfig
from copy import deepcopy

In [ ]:
torch.manual_seed(0)
# 超参数
beta = 0.1
# 加载模型
policy_model = LlamaForCausalLM(config=LlamaConfig(vocab_size=1000, num_hidden_layers=1, hidden_size=128))
reference_model = deepcopy(policy_model)

# data
prompt_ids = [1, 2, 3, 4, 5, 6]
good_response_ids = [7, 8, 9, 10]
# 对loss稍加修改可以应对一个good和多个bad的情况
bad_response_ids_list = [[1, 2, 3, 0], [4, 5, 6, 0]]

# 转换成模型输入
input_ids = torch.LongTensor(
    [prompt_ids + good_response_ids, *[prompt_ids + bad_response_ids for bad_response_ids in bad_response_ids_list]]
)

input_ids

"""
tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
        [ 1,  2,  3,  4,  5,  6,  1,  2,  3,  0],
        [ 1,  2,  3,  4,  5,  6,  4,  5,  6,  0]])
"""

In [ ]:
# labels 提前做个shift
labels = torch.LongTensor(
    [
        [-100] * len(prompt_ids) + good_response_ids,
        *[[-100] * len(prompt_ids) + bad_response_ids for bad_response_ids in bad_response_ids_list]
    ]
)[:, 1:]
labels

"""
tensor([[-100, -100, -100, -100, -100,    7,    8,    9,   10],
        [-100, -100, -100, -100, -100,    1,    2,    3,    0],
        [-100, -100, -100, -100, -100,    4,    5,    6,    0]])
"""

In [ ]:
loss_mask = (labels != -100)
labels[labels == -100] = 0

print(f"labels:{labels}\n")

print(f"loss_mask: {loss_mask}")


"""
labels:tensor([[ 0,  0,  0,  0,  0,  7,  8,  9, 10],
        [ 0,  0,  0,  0,  0,  1,  2,  3,  0],
        [ 0,  0,  0,  0,  0,  4,  5,  6,  0]])

loss_mask: tensor([[False, False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True,  True]])
"""

In [ ]:
print(f"labels shape: {labels.shape}")
labels.unsqueeze(2).shape

"""
labels shape: torch.Size([3, 9])
torch.Size([3, 9, 1])
"""

In [ ]:
logits = policy_model(input_ids)["logits"][:, :-1, :]
print(f"logits shape:{logits.shape}\n")

print(f"logits:\n{logits}")

In [ ]:
# 从词表中选取对应 label 位置上的概率值
per_token_logps = torch.gather(logits.log_softmax(-1), dim=2, index=labels.unsqueeze(2)).squeeze(2)
print(f"per_token_logps.shape:{per_token_logps.shape}")

print(f"per_token_logps:\n{per_token_logps}")

In [ ]:
# 选取对应 label 位置上概率求和
all_logps = (per_token_logps * loss_mask).sum(-1)
all_logps

In [ ]:
import torch
import torch.nn.functional as F
from transformers import LlamaForCausalLM, LlamaConfig
from copy import deepcopy

torch.manual_seed(0)
if __name__ == "__main__":
    # 超参数
    beta = 0.1
    # 加载模型
    policy_model = LlamaForCausalLM(config=LlamaConfig(vocab_size=1000, num_hidden_layers=1, hidden_size=128))
    reference_model = deepcopy(policy_model)

    # data
    prompt_ids = [1, 2, 3, 4, 5, 6]
    good_response_ids = [7, 8, 9, 10]
    # 对loss稍加修改可以应对一个good和多个bad的情况
    bad_response_ids_list = [[1, 2, 3, 0], [4, 5, 6, 0]]

    # 转换成模型输入
    input_ids = torch.LongTensor(
        [prompt_ids + good_response_ids, *[prompt_ids + bad_response_ids for bad_response_ids in bad_response_ids_list]]
    )
    # labels 提前做个shift
    labels = torch.LongTensor(
        [
            [-100] * len(prompt_ids) + good_response_ids,
            *[[-100] * len(prompt_ids) + bad_response_ids for bad_response_ids in bad_response_ids_list]
        ]
    )[:, 1:]
    loss_mask = (labels != -100)
    labels[labels == -100] = 0
    # 计算 policy model的log prob
    logits = policy_model(input_ids)["logits"][:, :-1, :]
    per_token_logps = torch.gather(logits.log_softmax(-1), dim=2, index=labels.unsqueeze(2)).squeeze(2)
    all_logps = (per_token_logps * loss_mask).sum(-1)
    # 暂时写死第一个是good response的概率
    policy_good_logps, policy_bad_logps = all_logps[:1], all_logps[1:]

    # 计算 reference model的log prob
    with torch.no_grad():
        logits = reference_model(input_ids)["logits"][:, :-1, :]
        per_token_logps = torch.gather(logits.log_softmax(-1), dim=2, index=labels.unsqueeze(2)).squeeze(2)
        all_logps = (per_token_logps * loss_mask).sum(-1)
        # 暂时写死第一个是good response的概率
        reference_good_logps, reference_bad_logps = all_logps[:1], all_logps[1:]

    # 计算loss，会自动进行广播
    logits = (policy_good_logps - reference_good_logps) - (policy_bad_logps - reference_bad_logps)
    loss = -F.logsigmoid(beta * logits).mean()
    print(loss)